In [ ]:
import cv2
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import cvlib as cv
import time
import os

# Function to detect motion and objects in a video
def detect_motion_and_objects(video_path, output_path, camera_mode=False):
    if not camera_mode:
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            messagebox.showerror("Error", "Could not open video file.")
            return
    else:
        cap = cv2.VideoCapture(0)  # Use the default camera (change the index if needed)

    # Get video frame properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(5))

    prev_frame = None
    motion_detected = False
    video_writer = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert frame to grayscale for motion detection
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_frame is not None:
            # Calculate the absolute difference in the frame
            diff = cv2.absdiff(gray_frame, prev_frame)

            # Check for motion in the frame
            if cv2.countNonZero(diff) > 1000:  # You can adjust this threshold
                motion_detected = True

                # Create a video writer if not already created
                if video_writer is None:
                    fourcc = cv2.VideoWriter_fourcc(*'XVID')
                    timestamp = int(time.time())
                    output_file = f"{output_path}/motion_capture_{timestamp}.avi"
                    video_writer = cv2.VideoWriter(output_file, fourcc, fps, (frame_width, frame_height))

                # Object detection using cvlib
                bbox, labels, conf = cv.detect_common_objects(frame)
                for label, conf_level, (x1, y1, x2, y2) in zip(labels, conf, bbox):
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f"{label}: {conf_level:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Save the frame with objects highlighted
                video_writer.write(frame)

        # Display "Motion Detected" on the frame if motion is detected
        if motion_detected:
            cv2.putText(frame, "Motion Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Update the label with the current frame
        update_label(frame)

        if motion_detected and not camera_mode:
            # Write the frame to the output video for video file input
            video_writer.write(frame)

        prev_frame = gray_frame

    if video_writer is not None:
        video_writer.release()

    cap.release()

# Function to update the label with the current frame
def update_label(frame):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_pil = Image.fromarray(frame_rgb)
    frame_tk = ImageTk.PhotoImage(image=frame_pil)
    video_label.config(image=frame_tk)
    video_label.image = frame_tk
    root.update_idletasks()  # Update the Tkinter window

# Function to start motion detection
def start_motion_detection():
    video_path = selected_video.get()
    output_path = output_directory.get()

    if not video_path and not camera_mode.get():
        messagebox.showerror("Error", "Please select a video file or enable camera mode.")
        return

    if not output_path:
        messagebox.showerror("Error", "Please select an output directory.")
        return

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    # Hide the input fields and button
    select_button.grid_forget()
    camera_mode_checkbox.grid_forget()
    output_directory_label.grid_forget()
    output_directory_entry.grid_forget()
    output_directory_button.grid_forget()

    # Reconfigure the video label to expand
    video_label.pack(fill=tk.BOTH, expand=True)

    # Start motion detection
    detect_motion_and_objects(video_path, output_path, camera_mode.get())

# Function to browse and select a video file
def browse_video():
    file_path = filedialog.askopenfilename(filetypes=[("Video Files", "*.mp4 *.avi *.mov")])
    if file_path:
        selected_video.set(file_path)
        camera_mode.set(False)  # Disable camera mode when selecting a video file
        
# Function to browse and select an output directory
def browse_output_directory():
    directory = filedialog.askdirectory()
    if directory:
        output_directory.set(directory)

# Create the main application window
root = tk.Tk()
root.title("Motion Detection with Object Detection (cvlib)")

# Maximize the main window across different platforms
root.state('zoomed')

# Create a label to display the video feed
video_label = tk.Label(root)
video_label.pack(fill=tk.BOTH, expand=True)

# Create an entry field for the output directory
output_directory = tk.StringVar()
output_directory_label = tk.Label(root, text="Output Directory:")
output_directory_label.pack(pady=5)
output_directory_entry = tk.Entry(root, textvariable=output_directory)
output_directory_entry.pack(pady=5)
output_directory_button = tk.Button(root, text="Browse", command=browse_output_directory)
output_directory_button.pack(pady=5)

# Create a button to select a video file
select_button = tk.Button(root, text="Select Video", command=browse_video, width=20, height=2)
select_button.pack(pady=10)

# Create a label to display the selected video file
selected_video = tk.StringVar()
selected_video_label = tk.Label(root, textvariable=selected_video)
selected_video_label.pack()

# Create a check button for enabling camera mode
camera_mode = tk.BooleanVar()
camera_mode.set(False)
camera_mode_checkbox = tk.Checkbutton(root, text="Live Camera Mode", variable=camera_mode)
camera_mode_checkbox.pack()

# Create a button to start motion detection
start_button = tk.Button(root, text="Start Detection", command=start_motion_detection, width=20, height=2)
start_button.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()
